# DS9-353 Toronto Postal Clusters

# Explore and Cluster Toronto Neighbourhoods

## Task
Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:

to add enough Markdown cells to explain what you decided to do and to report any observations you make.

to generate maps to visualize your neighborhoods and how they cluster together.

Once you are happy with your analysis, submit a link to the new Notebook on your Github repository. (3 marks)

In [104]:
tor_merged.loc[tor_merged['Cluster Labels'] == 2, tor_merged.columns[[2] + list(range(5, tor_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Islington Avenue,2,Grocery Store,Japanese Restaurant,Bus Stop,Bakery,Golf Course,Spa,Liquor Store,Café,Shopping Mall,Bank
6,"Rouge, Malvern",2,Zoo Exhibit,Fast Food Restaurant,Pizza Place,Spa,Chinese Restaurant,Big Box Store,Paper / Office Supplies Store,Fruit & Vegetable Store,Women's Store,Coffee Shop
8,"Woodbine Gardens, Parkview Hill",2,Pharmacy,Fast Food Restaurant,Park,Pizza Place,Intersection,Brewery,Skating Rink,Gym / Fitness Center,BBQ Joint,Rock Climbing Spot
11,"Cloverdale, Islington, Martin Grove, Princess ...",2,Park,Bank,Hotel,Recreation Center,Gym,Gym / Fitness Center,Electronics Store,Skating Rink,Theater,Clothing Store
12,"Highland Creek, Rouge Hill, Port Union",2,Park,Grocery Store,Pizza Place,Burger Joint,Neighborhood,Gym,Gym / Fitness Center,Italian Restaurant,Breakfast Spot,Playground
17,"Bloordale Gardens, Eringate, Markland Wood, Ol...",2,Baseball Field,Convenience Store,Bank,Grocery Store,Pizza Place,Coffee Shop,Liquor Store,Discount Store,Sandwich Place,Donut Shop
28,"Bathurst Manor, Downsview North, Wilson Heights",2,Park,Pizza Place,Coffee Shop,Building,Gas Station,Sushi Restaurant,Supermarket,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
39,Bayview Village,2,Bank,Trail,Grocery Store,Convenience Store,Chinese Restaurant,Intersection,Fast Food Restaurant,Japanese Restaurant,Park,Café
40,"CFB Toronto, Downsview East",2,Gym / Fitness Center,Racetrack,Athletics & Sports,Vietnamese Restaurant,Turkish Restaurant,Coffee Shop,Soccer Field,Basketball Court,Beer Store,Latin American Restaurant
46,Downsview West,2,Park,Bank,Tea Room,Grocery Store,Plaza,Moving Target,Pizza Place,Coffee Shop,Vietnamese Restaurant,Event Space


## Install libraries

In [60]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## Get and prep data

In [61]:
#read 'tor.csv' stored from previous notebook into pandas DataFrame 'tor'

with open('tor.csv') as tor:
    df_tor = pd.read_csv(tor, index_col='Unnamed: 0') #set indexcolumn from csv

print('The dataframe has {} boroughs and {} postal codes.'.format(
        len(df_tor['Borough'].unique()),
        df_tor.shape[0]
    )
)    
    
df_tor.head()

The dataframe has 11 boroughs and 103 postal codes.


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


#### Use geopy library to get the latitude and longitude values of Toronto.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>tor_explorer</em>, as shown below.

In [12]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.653963, -79.387207.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [16]:
# create map of Toronto using latitude and longitude values
map_tor = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_tor['Latitude'], df_tor['Longitude'], df_tor['Borough'], df_tor['Neighborhood']):
    label = '{} - {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tor)  
    
map_tor

## Use the Foursquare API to explore the neighborhoods and segment them

Define Foursquare Credentials and Version

In [21]:
# @hidden_cell
CLIENT_ID = 'O3PMPPQT1ZAH4SNIUCTHM3IS3TYGDGO51XELXSYNTZIBYWEF' # my Foursquare ID
CLIENT_SECRET = 'JF1HQCPLDORNOPSW1RODUWMKBPJ5B12DABCL3WEIWLHRMR5M' # my Foursquare Secret

In [43]:
#my Foursquare credentials: 'CLIENT_ID', 'CLIENT_SECRET' are defined elsewhwere for security
VERSION = '20180605' # Foursquare API version
LIMIT = 150

print('My credentails defined')

My credentails defined


#### Create a function to get nearby venues for all the neighborhoods in Toronto

In [55]:
def getNearbyVenues(names, latitudes, longitudes, radius=1500):
    
    num_ct=1
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name,num_ct)
        num_ct=num_ct+1
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Run the function on each neighborhood and create a new dataframe called tor_venues.

In [56]:
tor_venues = getNearbyVenues(names=df_tor['Neighborhood'],
                                   latitudes=df_tor['Latitude'],
                                   longitudes=df_tor['Longitude']
                                  )

print(tor_venues.shape)
tor_venues.head()

Parkwoods 1
Victoria Village 2
Harbourfront, Regent Park 3
Lawrence Heights, Lawrence Manor 4
Queen's Park 5
Islington Avenue 6
Rouge, Malvern 7
Don Mills North 8
Woodbine Gardens, Parkview Hill 9
Ryerson, Garden District 10
Glencairn 11
Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park 12
Highland Creek, Rouge Hill, Port Union 13
Flemingdon Park, Don Mills South 14
Woodbine Heights 15
St. James Town 16
Humewood-Cedarvale 17
Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe 18
Guildwood, Morningside, West Hill 19
The Beaches 20
Berczy Park 21
Caledonia-Fairbanks 22
Woburn 23
Leaside 24
Central Bay Street 25
Christie 26
Cedarbrae 27
Hillcrest Village 28
Bathurst Manor, Downsview North, Wilson Heights 29
Thorncliffe Park 30
Adelaide, King, Richmond 31
Dovercourt Village, Dufferin 32
Scarborough Village 33
Fairview, Henry Farm, Oriole 34
Northwood Park, York University 35
East Toronto 36
Harbourfront East, Toronto Islands, Union Station 37
Little Portugal,

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
2,Parkwoods,43.753259,-79.329656,Donalda Golf & Country Club,43.752816,-79.342741,Golf Course
3,Parkwoods,43.753259,-79.329656,Tim Hortons,43.760668,-79.326368,Café
4,Parkwoods,43.753259,-79.329656,LCBO,43.757774,-79.314257,Liquor Store


#### Let's find out how many unique categories can be curated from all the returned venues

In [57]:
print('There are {} uniques categories.'.format(len(tor_venues['Venue Category'].unique())))
print('There are {} hoods.'.format(len(tor_venues['Neighborhood'].unique())))

There are 338 uniques categories.
There are 103 hoods.


## Analyze Each Neighborhood

In [58]:
# one hot encoding
tor_onehot = pd.get_dummies(tor_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tor_onehot['Neighbourhood'] = tor_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [tor_onehot.columns[-1]] + list(tor_onehot.columns[:-1])
tor_onehot = tor_onehot[fixed_columns]

print(tor_onehot.shape)

tor_onehot.head()

(6786, 339)


,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Hungarian Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Medical Supply Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,National Park,Neighborhood,New American Restaurant,Nightclub,Noodle House,Nudist Beach,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Post Office,Poutine Place,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shanghai Restaura

#### Let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [62]:
tor_grouped = tor_onehot.groupby('Neighbourhood').mean().reset_index()

print(tor_grouped.shape)

tor_grouped.head()

(103, 339)


,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Hungarian Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Medical Supply Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,National Park,Neighborhood,New American Restaurant,Nightclub,Noodle House,Nudist Beach,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Post Office,Poutine Place,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shanghai Restaura

#### Let's write a function to sort the venues in descending order.

In [63]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [75]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = tor_grouped['Neighbourhood']

for ind in np.arange(tor_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tor_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Hotel,Theater,American Restaurant,Pizza Place,Japanese Restaurant,Gastropub,Thai Restaurant,Burrito Place
1,Agincourt,Chinese Restaurant,Coffee Shop,Asian Restaurant,Shopping Mall,Hong Kong Restaurant,Bakery,Japanese Restaurant,Park,Breakfast Spot,Gym / Fitness Center
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Chinese Restaurant,Bakery,Korean Restaurant,Coffee Shop,Dessert Shop,Pizza Place,Bubble Tea Shop,Pharmacy,BBQ Joint,Noodle House
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Fast Food Restaurant,Coffee Shop,Grocery Store,Pizza Place,Sandwich Place,Clothing Store,Flea Market,Greek Restaurant,Café,Beer Store
4,"Alderwood, Long Branch",Burger Joint,Coffee Shop,Pizza Place,Toy / Game Store,Pharmacy,Grocery Store,Café,Park,Restaurant,Burrito Place


## Cluster Neighborhoods

#### Run k-means to cluster the neighborhood into 5 clusters.

In [94]:
# set number of clusters
kclusters = 5

tor_grouped_clustering = tor_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tor_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 4, 4, 1, 0, 2, 2, 0, 0, 2], dtype=int32)

#### Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [95]:
# add clustering labels
neighborhoods_venues_sorted.drop('Cluster Labels', axis=1, inplace=True)
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

tor_merged = df_tor

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
tor_merged = tor_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighborhood')

tor_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1,Coffee Shop,Bus Stop,Pharmacy,Bank,Park,Pizza Place,Caribbean Restaurant,BBQ Joint,Office,Sandwich Place
1,M4A,North York,Victoria Village,43.725882,-79.315572,1,Coffee Shop,Gym,Fast Food Restaurant,Grocery Store,Middle Eastern Restaurant,Thrift / Vintage Store,Gym / Fitness Center,Chinese Restaurant,Clothing Store,Supermarket
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,0,Coffee Shop,Café,Park,Breakfast Spot,Bakery,Italian Restaurant,Restaurant,Farmers Market,Thai Restaurant,Theater
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,1,Clothing Store,Coffee Shop,Fast Food Restaurant,Sandwich Place,Furniture / Home Store,Dessert Shop,Vietnamese Restaurant,Restaurant,American Restaurant,Turkish Restaurant
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494,0,Coffee Shop,Japanese Restaurant,Gastropub,Bubble Tea Shop,Park,Restaurant,Ramen Restaurant,Pizza Place,Tea Room,Italian Restaurant


#### Let's visualize the resulting clusters

In [96]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tor_merged['Latitude'], tor_merged['Longitude'], tor_merged['Neighborhood'], tor_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

Now, let's examine each cluster and determine the discriminating venue categories that distinguish each cluster.

### Cluster 0

In [100]:
tor_merged.loc[tor_merged['Cluster Labels'] == 0, tor_merged.columns[[2] + list(range(5, tor_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"Harbourfront, Regent Park",0,Coffee Shop,Café,Park,Breakfast Spot,Bakery,Italian Restaurant,Restaurant,Farmers Market,Thai Restaurant,Theater
4,Queen's Park,0,Coffee Shop,Japanese Restaurant,Gastropub,Bubble Tea Shop,Park,Restaurant,Ramen Restaurant,Pizza Place,Tea Room,Italian Restaurant
7,Don Mills North,0,Coffee Shop,Japanese Restaurant,Restaurant,Burger Joint,Park,Italian Restaurant,Bank,Pizza Place,Ice Cream Shop,Asian Restaurant
9,"Ryerson, Garden District",0,Coffee Shop,Italian Restaurant,American Restaurant,Gastropub,Japanese Restaurant,Ramen Restaurant,Cosmetics Shop,Restaurant,Tea Room,Diner
13,"Flemingdon Park, Don Mills South",0,Coffee Shop,Gym,Japanese Restaurant,Sandwich Place,Park,Middle Eastern Restaurant,Restaurant,Italian Restaurant,Beer Store,Movie Theater
15,St. James Town,0,Café,Coffee Shop,Restaurant,Gastropub,Italian Restaurant,American Restaurant,Cosmetics Shop,Seafood Restaurant,Vegetarian / Vegan Restaurant,Steakhouse
16,Humewood-Cedarvale,0,Coffee Shop,Italian Restaurant,Pizza Place,Bank,Café,Sandwich Place,Sushi Restaurant,Pharmacy,Bakery,Caribbean Restaurant
19,The Beaches,0,Coffee Shop,Pub,Japanese Restaurant,Breakfast Spot,Grocery Store,Bakery,Bar,Beach,Sandwich Place,Bookstore
20,Berczy Park,0,Coffee Shop,Café,Restaurant,Park,Hotel,Japanese Restaurant,Italian Restaurant,Gastropub,Seafood Restaurant,Beer Bar
21,Caledonia-Fairbanks,0,Italian Restaurant,Coffee Shop,Pizza Place,Café,Furniture / Home Store,Caribbean Restaurant,Bakery,Grocery Store,Bank,Fried Chicken Joint


### Cluster 1

In [103]:
tor_merged.loc[tor_merged['Cluster Labels'] == 1, tor_merged.columns[[2] + list(range(5, tor_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Parkwoods,1,Coffee Shop,Bus Stop,Pharmacy,Bank,Park,Pizza Place,Caribbean Restaurant,BBQ Joint,Office,Sandwich Place
1,Victoria Village,1,Coffee Shop,Gym,Fast Food Restaurant,Grocery Store,Middle Eastern Restaurant,Thrift / Vintage Store,Gym / Fitness Center,Chinese Restaurant,Clothing Store,Supermarket
3,"Lawrence Heights, Lawrence Manor",1,Clothing Store,Coffee Shop,Fast Food Restaurant,Sandwich Place,Furniture / Home Store,Dessert Shop,Vietnamese Restaurant,Restaurant,American Restaurant,Turkish Restaurant
10,Glencairn,1,Fast Food Restaurant,Grocery Store,Coffee Shop,Bank,Park,Caribbean Restaurant,Pizza Place,Sandwich Place,Playground,Restaurant
14,Woodbine Heights,1,Pizza Place,Coffee Shop,Thai Restaurant,Pharmacy,Park,Sandwich Place,Breakfast Spot,Gastropub,Cosmetics Shop,Convenience Store
18,"Guildwood, Morningside, West Hill",1,Pizza Place,Breakfast Spot,Fast Food Restaurant,Coffee Shop,Smoothie Shop,Asian Restaurant,Sandwich Place,Beer Store,Supermarket,Bank
22,Woburn,1,Coffee Shop,Pizza Place,Fast Food Restaurant,Indian Restaurant,Pharmacy,Chinese Restaurant,Beer Store,Juice Bar,Music Store,Clothing Store
26,Cedarbrae,1,Coffee Shop,Clothing Store,Indian Restaurant,Sandwich Place,Gym / Fitness Center,Restaurant,Wings Joint,Fast Food Restaurant,Bakery,Pharmacy
29,Thorncliffe Park,1,Coffee Shop,Sandwich Place,Indian Restaurant,Restaurant,Convenience Store,Sporting Goods Shop,Burger Joint,Grocery Store,Electronics Store,Pharmacy
32,Scarborough Village,1,Sandwich Place,Fast Food Restaurant,Grocery Store,Pharmacy,Breakfast Spot,Coffee Shop,Chinese Restaurant,Pizza Place,Wings Joint,Liquor Store


### Cluster 2

In [105]:
tor_merged.loc[tor_merged['Cluster Labels'] == 2, tor_merged.columns[[2] + list(range(5, tor_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Islington Avenue,2,Grocery Store,Japanese Restaurant,Bus Stop,Bakery,Golf Course,Spa,Liquor Store,Café,Shopping Mall,Bank
6,"Rouge, Malvern",2,Zoo Exhibit,Fast Food Restaurant,Pizza Place,Spa,Chinese Restaurant,Big Box Store,Paper / Office Supplies Store,Fruit & Vegetable Store,Women's Store,Coffee Shop
8,"Woodbine Gardens, Parkview Hill",2,Pharmacy,Fast Food Restaurant,Park,Pizza Place,Intersection,Brewery,Skating Rink,Gym / Fitness Center,BBQ Joint,Rock Climbing Spot
11,"Cloverdale, Islington, Martin Grove, Princess ...",2,Park,Bank,Hotel,Recreation Center,Gym,Gym / Fitness Center,Electronics Store,Skating Rink,Theater,Clothing Store
12,"Highland Creek, Rouge Hill, Port Union",2,Park,Grocery Store,Pizza Place,Burger Joint,Neighborhood,Gym,Gym / Fitness Center,Italian Restaurant,Breakfast Spot,Playground
17,"Bloordale Gardens, Eringate, Markland Wood, Ol...",2,Baseball Field,Convenience Store,Bank,Grocery Store,Pizza Place,Coffee Shop,Liquor Store,Discount Store,Sandwich Place,Donut Shop
28,"Bathurst Manor, Downsview North, Wilson Heights",2,Park,Pizza Place,Coffee Shop,Building,Gas Station,Sushi Restaurant,Supermarket,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
39,Bayview Village,2,Bank,Trail,Grocery Store,Convenience Store,Chinese Restaurant,Intersection,Fast Food Restaurant,Japanese Restaurant,Park,Café
40,"CFB Toronto, Downsview East",2,Gym / Fitness Center,Racetrack,Athletics & Sports,Vietnamese Restaurant,Turkish Restaurant,Coffee Shop,Soccer Field,Basketball Court,Beer Store,Latin American Restaurant
46,Downsview West,2,Park,Bank,Tea Room,Grocery Store,Plaza,Moving Target,Pizza Place,Coffee Shop,Vietnamese Restaurant,Event Space


### Cluster 3

In [106]:
tor_merged.loc[tor_merged['Cluster Labels'] == 3, tor_merged.columns[[2] + list(range(5, tor_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
95,Upper Rouge,3,Farm,National Park,Zoo Exhibit,Fast Food Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant


### Cluster 4

In [107]:
tor_merged.loc[tor_merged['Cluster Labels'] == 4, tor_merged.columns[[2] + list(range(5, tor_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,Hillcrest Village,4,Chinese Restaurant,Coffee Shop,Park,Pharmacy,Bakery,Bank,Sandwich Place,Supermarket,Sushi Restaurant,Pizza Place
78,Agincourt,4,Chinese Restaurant,Coffee Shop,Asian Restaurant,Shopping Mall,Hong Kong Restaurant,Bakery,Japanese Restaurant,Park,Breakfast Spot,Gym / Fitness Center
85,"Agincourt North, L'Amoreaux East, Milliken, St...",4,Chinese Restaurant,Bakery,Korean Restaurant,Coffee Shop,Dessert Shop,Pizza Place,Bubble Tea Shop,Pharmacy,BBQ Joint,Noodle House
90,L'Amoreaux West,4,Chinese Restaurant,Bakery,Fast Food Restaurant,Coffee Shop,Sandwich Place,Pizza Place,Pool,Tennis Court,Grocery Store,Caribbean Restaurant


#### Cluster 0
Predominantly coffee shops, varied restaurants, bars/pubs, some parks around
#### Cluster 1
Many coffee shops and restaurants around but can also find some shopping spots
#### Cluster 2
Varied venues, including restaurants, grocery, pharmacy, parks, other places for activities (stadiums etc)
#### Cluster 3
A cluster of one: farms, national park, zoo
#### Cluster 4
Most common venue is a Chinese restaurant